# German Textsummary CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time

## Params

In [2]:
BATCH_SIZE = 10

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [3]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

In [4]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275392 
Total params: 222,903,936
Trainable params: 222,903,936
Non-trainable params: 0
_________________________________________________________________


In [5]:
ckpt_file = "../models/ckpt/checkpoint.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [6]:
path = "../data"

def get_tfrecord_dataset(drive_path, file_name):
    features = {
        'x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN], tf.int64),
        'x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN], tf.int64),
        'y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
        'y_labels': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
    }

    dataset = tf.data.TFRecordDataset(f"{drive_path}/{file_name}.tfrecord")

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        return record['x'], record['x_mask'], record['y'], record['y_ids'], record['y_labels']


    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(100)
    return dataset.batch(BATCH_SIZE)

test_ds = get_tfrecord_dataset(path, "test_cnn_daily_mail")

## Evaluation
### Define Rouge Score

In [7]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Rouge Score

In [8]:
predictions = []
rouge_score = RougeScore()
start_time = time.time()

for i, (input_ids, input_mask, y, y_ids, y_labels) in enumerate(test_ds):   
    summaries = model.generate(
        input_ids=input_ids, 
        attention_mask=input_mask, 
        early_stopping=True, 
        max_length=150, 
        min_length=40, 
        do_sample= True, 
        length_penalty=100.0,
        repetition_penalty=2.0,
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1
    )

    articles = [tokenizer.decode(g[1:], skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]
    
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
    for pred_sent, real_sent, article_sent in zip(pred, real, articles):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    break
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 51:
        # otherwise it will take ages
        break


# rouge_score.result()

### Lets have a look at some of these predicted summaries

In [9]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [10]:
print(get_random_prediction())

article: : Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England-Rundfahrt in den Westindien ein dringend benötigtes Jahrhundert. Cook wurde 95 und erreichte mit minimalem Aufhebens dreistellige Werte, bevor er in den Ruhestand ging. Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England Tour ein Jahrhundert. Eine kontrollierte dicke Flanke vom ersten Ball des Tages brachte ihm eine 11. Grenze der Innings und in der Folge drosch er den Ball für zwei vom hinteren Fuß. Er winkte seinem Partner Gary Ballance sanft mit dem Schläger und gab ihm einen Händedruck und ging auf 101 von 200 Lieferungen los. Das ermöglichte Ian Bell, an den Knick zu gelangen, wobei die Schlagzeit für die Touristen wichtiger war als der Versuch, ein Ergebnis in dieser zweitägigen Veranstaltung zu erzwingen. Ian Bell steht am zweiten Tag des St. Kitts und Nevis Invitational XI gegen England im Abseits.

pred sentence: retetander Kapitän Al

In [11]:
print(get_random_prediction())

article: : Prinzessin Beatrice wurde beim Großen Preis von Bahrain gesehen, ihrem vierten Urlaub in einem Monat (im Bild beim Großen Preis von Abu Dhabi im vergangenen November). Nach elf Urlauben in weniger als sechs Monaten hätte man gedacht, Prinzessin Beatrice wäre bereit, ihren Weltenbummler zu bremsen. Doch gestern wurde die unterbeschäftigte 26-Jährige wieder einmal in Übersee gesichtet, diesmal beim Großen Preis von Bahrain. Sie befand sich im Golfstaat, regiert von einem umstrittenen König, dessen Regime beschuldigt wird, pro-demokratische Aktivisten gewaltsam zu unterdrücken, laut einem Zuschauer, der sie vor dem Rennen unter Würdenträgern in der Startaufstellung sah. Sie ging hinter dem Kronprinzen von Bahrain her. Sie sei dem kürzlich abgedankten König Carlos von Spanien vorgestellt und beschnitten worden, sagte die Quelle. Angeführt wurde diese Gruppe von Würdenträgern von Bernie Ecclestone. Ich glaube, sie hat das Rennen mit dem Kronprinzen vom Turm hoch über der Rennstre

In [12]:
print(get_random_prediction())

article: : Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England-Rundfahrt in den Westindien ein dringend benötigtes Jahrhundert. Cook wurde 95 und erreichte mit minimalem Aufhebens dreistellige Werte, bevor er in den Ruhestand ging. Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England Tour ein Jahrhundert. Eine kontrollierte dicke Flanke vom ersten Ball des Tages brachte ihm eine 11. Grenze der Innings und in der Folge drosch er den Ball für zwei vom hinteren Fuß. Er winkte seinem Partner Gary Ballance sanft mit dem Schläger und gab ihm einen Händedruck und ging auf 101 von 200 Lieferungen los. Das ermöglichte Ian Bell, an den Knick zu gelangen, wobei die Schlagzeit für die Touristen wichtiger war als der Versuch, ein Ergebnis in dieser zweitägigen Veranstaltung zu erzwingen. Ian Bell steht am zweiten Tag des St. Kitts und Nevis Invitational XI gegen England im Abseits.

pred sentence: retetander Kapitän Al

In [13]:
print(get_random_prediction())

article: : Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England-Rundfahrt in den Westindien ein dringend benötigtes Jahrhundert. Cook wurde 95 und erreichte mit minimalem Aufhebens dreistellige Werte, bevor er in den Ruhestand ging. Englands Kapitän Alastair Cook vollendete am zweiten Morgen des Eröffnungsspiels der England Tour ein Jahrhundert. Eine kontrollierte dicke Flanke vom ersten Ball des Tages brachte ihm eine 11. Grenze der Innings und in der Folge drosch er den Ball für zwei vom hinteren Fuß. Er winkte seinem Partner Gary Ballance sanft mit dem Schläger und gab ihm einen Händedruck und ging auf 101 von 200 Lieferungen los. Das ermöglichte Ian Bell, an den Knick zu gelangen, wobei die Schlagzeit für die Touristen wichtiger war als der Versuch, ein Ergebnis in dieser zweitägigen Veranstaltung zu erzwingen. Ian Bell steht am zweiten Tag des St. Kitts und Nevis Invitational XI gegen England im Abseits.

pred sentence: retetander Kapitän Al

## Upsides
The results are good readable and they summaries the content of the article.

## Downsides
A obvious fault of the model is, that it does not stop generating. It's maybe caused by the way the model was trained. The length penalty does not have any effect. If 

In [14]:
result_path = "../data/t5base_result_german.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")